# **Cargado de archivos**

In [4]:
# =============================================================================
# Configuración optimizada de la sesión de Spark
# =============================================================================
# Esta configuración es más robusta para conjuntos de datos grandes.
# Debe ejecutarse una vez al inicio del notebook.

from pyspark.sql import SparkSession
import os

spark = SparkSession.builder \
    .appName("BlueBikes-Project") \
    .config("spark.driver.memory", "8g") \
    .config("spark.executor.memory", "8g") \
    .config("spark.sql.shuffle.partitions", "200") \
    .config("spark.network.timeout", "800s") \
    .config("spark.executor.heartbeatInterval", "60s") \
    .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
    .config("spark.driver.maxResultSize", "2g") \
    .getOrCreate()

# Establecer variables de entorno para que PySpark funcione correctamente en algunos entornos
import sys
# os.environ["PYSPARK_PYTHON"] = sys.executable
# os.environ["PYSPARK_DRIVER_PYTHON"] = sys.executable
os.environ["PYSPARK_PYTHON"] = "python"
os.environ["PYSPARK_DRIVER_PYTHON"] = "python"

In [5]:
# =============================================================================
# Cargar DataFrame procesado desde Google Drive (para Colab)
# =============================================================================
# Esta celda activa tu Google Drive y lee los datos procesados del archivo Parquet almacenado allí.
# Usarlo en el entorno de Google Colab.

from google.colab import drive

print("Intentando montar Google Drive...")
try:
    drive.mount('/content/drive')

    gdrive_path = "/content/drive/MyDrive/DM_PRJ/df_final_bluebikes_v2.parquet"
    print(f"Cargando datos desde la ruta de Google Drive: {gdrive_path}")

    # Spark leerá la carpeta Parquet directamente
    df_final = spark.read.parquet(gdrive_path)

    print("✅ DataFrame cargado exitosamente desde Google Drive.")

    # Verify the schema and show a few rows
    print("Esquema de DataFrame:")
    df_final.printSchema()

    print("Muestra de los datos cargados:")
    df_final.show(5, truncate=False)

except Exception as e:
    print(f"❌ Error al cargar datos desde Google Drive. Asegúrate de que el archivo exista en '{gdrive_path}'.")
    print(f"Detalles del error: {e}")

Intentando montar Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Cargando datos desde la ruta de Google Drive: /content/drive/MyDrive/DM_PRJ/df_final_bluebikes_v2.parquet
✅ DataFrame cargado exitosamente desde Google Drive.
Esquema de DataFrame:
root
 |-- start_station_id: string (nullable = true)
 |-- ride_id: string (nullable = true)
 |-- started_at: timestamp (nullable = true)
 |-- ended_at: timestamp (nullable = true)
 |-- start_station_name: string (nullable = true)
 |-- start_lat: double (nullable = true)
 |-- start_lng: double (nullable = true)
 |-- end_station_id: string (nullable = true)
 |-- end_station_name: string (nullable = true)
 |-- end_lat: double (nullable = true)
 |-- end_lng: double (nullable = true)
 |-- member_casual: string (nullable = true)
 |-- duration_sec: long (nullable = true)
 |-- schema_version: string (nullable = true)
 |-- periodo: string (nullable = true)


# **Evaluación de modelo**

In [10]:
# =============================================================================
# Selección avanzada de funciones (con conjunto de características refinado)
# =============================================================================

from pyspark.ml.feature import VectorAssembler, StringIndexer, OneHotEncoder
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml import Pipeline
from pyspark.sql.functions import col
import pandas as pd

# --- Paso 1: Preparación de datos ---

# Realizar un muestreo estratificado en la columna de cadena original.
label_col = 'member_casual'
distinct_labels = [row[label_col] for row in df_final.select(label_col).distinct().collect()]
fractions = {label: 0.5 for label in distinct_labels} # Usar una muestra del 50%

# Crear la muestra estratificada y divídala en conjuntos de entrenamiento y prueba
df_sample = df_final.sampleBy(label_col, fractions=fractions, seed=42)
train_data, test_data = df_sample.randomSplit([0.8, 0.2], seed=42)

# Almacenar en caché los datos de entrenamiento para un acceso más rápido
train_data.cache().count()

# --- Paso 2: Definir el pipeline completo ---

# 1. Indexar la columna de etiquetas
label_indexer = StringIndexer(inputCol=label_col, outputCol='label', handleInvalid='keep')

# 2. CAMBIO: Actualizar la lista de características numéricas
#    - Se AÑADEN 'avg_speed_kmh' y 'is_round_trip'.
#    - Se ELIMINAN 'trip_hour' y 'trip_day_of_week' para evitar redundancia con las versiones sin/cos.
categorical_features = ['season']
numerical_features = [
    'log_duration', 'haversine_distance_km', 'is_popular_start',
    'trip_month',
    'hour_sin', 'hour_cos', 'day_of_week_sin', 'day_of_week_cos',
    'avg_speed_kmh', 'is_round_trip' # <-- NUEVAS CARACTERÍSTICAS
]

# 3. Crear etapas para la codificación de características categóricas
stages = [label_indexer]
encoded_feature_names = []
for c in categorical_features:
    indexer = StringIndexer(inputCol=c, outputCol=f"{c}_indexed", handleInvalid='keep')
    encoder = OneHotEncoder(inputCol=f"{c}_indexed", outputCol=f"{c}_encoded", dropLast=False)
    stages.extend([indexer, encoder])
    encoded_feature_names.append(f"{c}_encoded")

# 4. Ensamblar todas las características en un solo vector
all_assembler_cols = numerical_features + encoded_feature_names
assembler = VectorAssembler(inputCols=all_assembler_cols, outputCol="features")
stages.append(assembler)

# 5. Definir el modelo RandomForest para obtener la importancia de las características
rf = RandomForestClassifier(labelCol='label', featuresCol="features", numTrees=100, seed=42)
stages.append(rf)

# Crear el objeto de pipeline completo
pipeline = Pipeline(stages=stages)

# --- Paso 3: Entrenar el modelo y extraer la importancia de las características ---

model = pipeline.fit(train_data)

# Extraer puntuaciones de importancia
importances = model.stages[-1].featureImportances.toArray()

# Obtener los nombres de las características
feature_names = model.stages[-2].getInputCols()

# Crear un DataFrame para mostrar la importancia de las características
importance_df = pd.DataFrame(list(zip(feature_names, importances)), columns=['feature', 'importance'])
importance_df = importance_df.sort_values('importance', ascending=False)

print("Las 10 características principales por importancia (con el nuevo conjunto de características):")
print(importance_df.head(10))

# Seleccionar dinámicamente las 10 características principales para el modelo final
top_features = importance_df.head(10)['feature'].tolist()
print(f"\nCaracterísticas seleccionadas para el modelo final:\n{top_features}")

Las 10 características principales por importancia (con el nuevo conjunto de características):
                  feature  importance
8           avg_speed_kmh    0.408620
0            log_duration    0.275662
7         day_of_week_cos    0.138945
9           is_round_trip    0.065822
1   haversine_distance_km    0.058860
3              trip_month    0.008754
4                hour_sin    0.006530
10         season_encoded    0.005388
2        is_popular_start    0.005151
6         day_of_week_sin    0.004390

Características seleccionadas para el modelo final:
['avg_speed_kmh', 'log_duration', 'day_of_week_cos', 'is_round_trip', 'haversine_distance_km', 'trip_month', 'hour_sin', 'season_encoded', 'is_popular_start', 'day_of_week_sin']


In [11]:
# ============================================================================================
# MOdelo Final: GBTClassifier con Hyperparameter Tuning utilizando CrossValidator
# ============================================================================================
# Este es el bloque final de modelado. Se utiliza un CrossValidator para encontrar los mejores
# hiperparámetros para el modelo GBT, asegurando el máximo rendimiento posible.

# CAMBIO: Importar las herramientas necesarias para la validación cruzada
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml import Pipeline, PipelineModel
from pyspark.mllib.evaluation import MulticlassMetrics
import pandas as pd
from pyspark.sql.functions import when, col

# --- Paso 1: Preparar datos (sin cambios) ---
feature_transformer_pipeline = PipelineModel(stages=model.stages[:-1])
transformed_train_data = feature_transformer_pipeline.transform(train_data)
transformed_test_data = feature_transformer_pipeline.transform(test_data)

# --- Paso 2: Calcular y aplicar pesos de clase (sin cambios) ---
total_count = train_data.count()
casual_count = train_data.filter(col("member_casual") == "casual").count()
member_count = total_count - casual_count
weight_casual = total_count / (2.0 * casual_count)
weight_member = total_count / (2.0 * member_count)
print(f"Peso calculado para la clase 'member': {weight_member:.2f}")
print(f"Peso calculado para la clase 'casual': {weight_casual:.2f}\n")
weighted_train_data = transformed_train_data.withColumn(
    "class_weight",
    when(col("member_casual") == "casual", weight_casual)
    .otherwise(weight_member)
)

# --- Paso 3: Configurar el Pipeline de Afinamiento (Hyperparameter Tuning) ---
print(f"Reconstruyendo el pipeline para afinamiento con las siguientes características:\n {top_features}\n")

# 1. Ensamblador de características (sin cambios)
final_assembler_cols = top_features
assembler_final = VectorAssembler(inputCols=final_assembler_cols, outputCol="final_features")

# 2. Definir el modelo base GBTClassifier
gbt = GBTClassifier(
    labelCol="label",
    featuresCol="final_features",
    weightCol="class_weight",
    seed=42
)

# 3. CAMBIO: Construir una grilla de hiperparámetros para probar
paramGrid = (ParamGridBuilder()
             .addGrid(gbt.maxDepth, [3, 5])      # Profundidad de los árboles
             .addGrid(gbt.maxIter, [20, 40])     # Número de árboles
             .build())

# 4. CAMBIO: Usar un evaluador binario, más adecuado para problemas de dos clases (member/casual)
#    La métrica 'areaUnderPR' (Área bajo la curva de Precisión-Recall) es excelente para datos desbalanceados.
evaluator = BinaryClassificationEvaluator(labelCol='label', rawPredictionCol='rawPrediction', metricName='areaUnderPR')

# 5. CAMBIO: Configurar el CrossValidator
#    Este se encargará de entrenar y evaluar múltiples versiones del modelo para encontrar la mejor.
crossval = CrossValidator(estimator=gbt,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=3)  # Usar 3 divisiones para un balance entre robustez y tiempo de ejecución

# 6. Crear el pipeline final. Su última etapa es el CrossValidator.
pipeline_final = Pipeline(stages=[assembler_final, crossval])


# --- Paso 4: Entrenamiento y Evaluación ---
# ¡Este paso ahora ejecutará todo el proceso de validación cruzada!
print("Iniciando el proceso de afinamiento con CrossValidator... Esto puede tardar varios minutos.")
final_model = pipeline_final.fit(weighted_train_data)
print("Afinamiento completado.")

# Realizar predicciones usando el mejor modelo encontrado por el CrossValidator
predictions = final_model.transform(transformed_test_data)


# --- Paso 5: Métricas de Rendimiento del Mejor Modelo ---
evaluator_accuracy = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName='accuracy')
accuracy = evaluator_accuracy.evaluate(predictions)

print(f"\nNueva precisión del modelo (GBT Afinaddo): {accuracy:.4f}")
print(f"Precisión anterior (GBT sin afinar): 0.6831")
improvement = accuracy - 0.6831
print(f"Mejora del modelo: {improvement:+.4f}\n")

# Get the label mapping from the trained pipeline model
# Access the StringIndexer model from the trained pipeline model
label_indexer_model = model.stages[0]
# Get the mapping of original labels to indexed labels
label_map = {index: label for index, label in enumerate(label_indexer_model.labels)}

# Métricas de la matriz de confusión
predictionAndLabels = predictions.select('prediction', 'label').rdd.map(lambda r: (float(r.prediction), float(r.label)))
metrics = MulticlassMetrics(predictionAndLabels)

confusion_matrix = metrics.confusionMatrix().toArray()
class_labels = model.stages[0].labels
report_df = pd.DataFrame(confusion_matrix,
                         index=[f'Actual: {l}' for l in class_labels],
                         columns=[f'Predicted: {l}' for l in class_labels])
print("Matriz de confusión del mejor modelo:")
print(report_df)

# Utilizar label_map para obtener el índice numérico de 'member'
positive_class_label = float([k for k, v in label_map.items() if v == 'member'][0])
print("\nInforme de clasificación del mejor modelo (para la clase 'member'):")
print(f"Precision: {metrics.precision(positive_class_label):.4f}")
print(f"Recall: {metrics.recall(positive_class_label):.4f}")
print(f"F1-Score: {metrics.fMeasure(positive_class_label):.4f}\n")

train_data.unpersist()

Peso calculado para la clase 'member': 0.64
Peso calculado para la clase 'casual': 2.28

Reconstruyendo el pipeline para afinamiento con las siguientes características:
 ['avg_speed_kmh', 'log_duration', 'day_of_week_cos', 'is_round_trip', 'haversine_distance_km', 'trip_month', 'hour_sin', 'season_encoded', 'is_popular_start', 'day_of_week_sin']

Iniciando el proceso de afinamiento con CrossValidator... Esto puede tardar varios minutos.
Afinamiento completado.

Nueva precisión del modelo (GBT Afinaddo): 0.6866
Precisión anterior (GBT sin afinar): 0.6831
Mejora del modelo: +0.0035



/usr/local/lib/python3.11/dist-packages/pyspark/sql/context.py:158: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


Matriz de confusión del mejor modelo:
                Predicted: member  Predicted: casual
Actual: member          1353461.0           564061.0
Actual: casual           205319.0           332351.0

Informe de clasificación del mejor modelo (para la clase 'member'):
Precision: 0.8683
Recall: 0.7058
F1-Score: 0.7787



DataFrame[start_station_id: string, ride_id: string, started_at: timestamp, ended_at: timestamp, start_station_name: string, start_lat: double, start_lng: double, end_station_id: string, end_station_name: string, end_lat: double, end_lng: double, member_casual: string, duration_sec: bigint, schema_version: string, periodo: string, log_duration: double, trip_year: int, trip_month: int, trip_day_of_week: int, trip_hour: int, is_weekend: int, season: string, hour_sin: double, hour_cos: double, day_of_week_sin: double, day_of_week_cos: double, haversine_distance_km: double, is_popular_start: int, avg_speed_kmh: double, is_round_trip: int]